# Chapter 5. Model evaluation and enhancement.
# Part 3. Quality metrics.
R^2 for regressions and accuracy for classificators sometimes are not the metrics that really needed. Thus there are more.

2 types of errors:

1) False-positive

2) False-negative

Accuracy is NOT an adequate metric for evaluating model prognostic ability fitted over unbalanced datasets.